In [1]:

import sys
import pandas as pd
import numpy as np


def read_file(file_name):
    with open(file_name+".txt") as file:
    #     kk=f.readlines()
        lines = file.readlines()
        rows1=[]
        # Process each line
        for line in lines:
            # Split values based on spaces
            values = line.split()

           
            i=0
            d={}
            for val in values:
                d[f'col{i}']=val
                i+=1
            rows1.append(d)

    df_yt=pd.DataFrame(rows1)
    df_yt.rename(columns={df_yt.columns[-1]:"label"},inplace=True)
    
    return df_yt

In [149]:
train_df=read_file("yeast_training")
train_df

,col0,col1,col2,col3,col4,col5,col6,col7,label
0,0.5000,0.4600,0.6400,0.3600,0.5000,0,0.4900,0.2200,1
1,0.5300,0.5600,0.4900,0.4600,0.5000,0,0.5200,0.2200,1
2,0.5200,0.5300,0.5800,0.6900,0.5000,0,0.5000,0.2200,1
3,0.6700,0.6200,0.5400,0.4300,0.5000,0,0.5300,0.2200,1
4,0.4500,0.5700,0.3000,0.1700,0.5000,0,0.5100,0.2200,3
...,...,...,...,...,...,...,...,...,...
995,0.7400,0.7200,0.5000,0.2800,0.5000,0,0.4900,0.2700,8
996,0.5900,0.6400,0.4400,0.2600,0.5000,0,0.5000,0.2200,7
997,0.8000,0.8300,0.4300,0.2400,0.5000,0,0.5400,0.2200,4
998,0.6000,0.6800,0.3000,0.2700,0.5000,0,0.5500,0.2200,3


In [150]:
test_df=read_file("yeast_test")
test_df

,col0,col1,col2,col3,col4,col5,col6,col7,label
0,0.5200,0.4600,0.3400,0.2000,0.5000,0,0.5100,0.2500,3
1,0.4000,0.4500,0.6100,0.2900,0.5000,0,0.4400,0.2200,7
2,0.4000,0.4700,0.4800,0.1800,0.5000,0,0.4600,0.2200,7
3,0.4600,0.3200,0.5800,0.1700,0.5000,0,0.6100,0.2500,2
4,0.3200,0.3400,0.6100,0.1700,0.5000,0,0.5200,0.3200,2
...,...,...,...,...,...,...,...,...,...
479,0.5400,0.4400,0.3800,0.2200,0.5000,0,0.5400,0.3000,3
480,0.4400,0.6200,0.5300,0.2700,0.5000,0,0.4900,0.2200,2
481,0.6600,0.6500,0.5400,0.4000,0.5000,0,0.5100,0.2200,8
482,0.2700,0.5100,0.4700,0.3500,0.5000,0,0.4900,0.2800,2


In [20]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.features = None  # Store the features used during training

    def _calculate_entropy(self, y):
        # Function to calculate entropy
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _calculate_information_gain(self, X, y, feature, threshold):
        # Function to calculate information gain
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_entropy = self._calculate_entropy(y[left_mask])
        right_entropy = self._calculate_entropy(y[right_mask])

        total_entropy = self._calculate_entropy(y)
        information_gain = total_entropy - (np.sum(left_mask) / len(y) * left_entropy +
                                           np.sum(right_mask) / len(y) * right_entropy)

        return information_gain

    def _find_best_split(self, X, y):
        # Function to find the best split for a node
        m, n = X.shape
        best_feature = None
        best_threshold = None
        max_information_gain = -np.inf

        for feature in range(n):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                information_gain = self._calculate_information_gain(X, y, feature, threshold)

                if information_gain > max_information_gain:
                    max_information_gain = information_gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _grow_tree(self, X, y, depth):
        # Recursive function to grow the tree
        unique_classes = np.unique(y)

        if len(unique_classes) == 1:
            # If pure node, return a leaf node
            return {'value': unique_classes[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            # If maximum depth reached, return a leaf node with majority class
            majority_class = np.argmax(np.bincount(y))
            return {'value': majority_class}

        best_feature, best_threshold = self._find_best_split(X, y)

        if best_feature is None:
            # If no split found, return a leaf node with majority class
            majority_class = np.argmax(np.bincount(y))
            return {'value': majority_class}

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    def fit(self, X, y):
        # Train the decision tree
        self.features = set(range(X.shape[1]))
        self.tree = self._grow_tree(X, y, depth=0)

    def _predict_instance(self, x, node):
        # Recursive function to predict a single instance
        if 'value' in node:
            return node['value']

        if x[node['feature']] <= node['threshold']:
            return self._predict_instance(x, node['left'])
        else:
            return self._predict_instance(x, node['right'])

    def predict(self, X):
        # Predict for multiple instances
        if set(range(X.shape[1])) != self.features:
            raise ValueError("The number or order of features in the test set does not match the training set.")

        return [self._predict_instance(x, self.tree) for x in X]


In [21]:
dt=DecisionTree()
dt.fit(X_train,y_train)

In [22]:
dt.predict(X_test)

[1, 1]

In [39]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None, random_features=False):
        self.max_depth = max_depth
        self.tree = None
        self.random_features = random_features
        self.features = None  # Store the features used during training

    def _calculate_entropy(self, y):
        # Function to calculate entropy
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _calculate_information_gain(self, X, y, feature, threshold):
        # Function to calculate information gain
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_entropy = self._calculate_entropy(y[left_mask])
        right_entropy = self._calculate_entropy(y[right_mask])

        total_entropy = self._calculate_entropy(y)
        information_gain = total_entropy - (np.sum(left_mask) / len(y) * left_entropy +
                                           np.sum(right_mask) / len(y) * right_entropy)

        return information_gain

    def _find_best_split(self, X, y):
        # Function to find the best split for a node
        m, n = X.shape
        best_feature = None
        best_threshold = None
        max_information_gain = -np.inf

        if self.random_features:
            features_subset = np.random.choice(n, int(np.sqrt(n)), replace=False)
        else:
            features_subset = range(n)

        for feature in features_subset:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                information_gain = self._calculate_information_gain(X, y, feature, threshold)

                if information_gain > max_information_gain:
                    max_information_gain = information_gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _grow_tree(self, X, y, depth):
        # Recursive function to grow the tree
        unique_classes = np.unique(y)

        if len(unique_classes) == 1:
            # If pure node, return a leaf node
            return {'value': unique_classes[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            # If maximum depth reached, return a leaf node with majority class
            majority_class = np.argmax(np.bincount(y))
            return {'value': majority_class}

        best_feature, best_threshold = self._find_best_split(X, y)

        if best_feature is None:
            # If no split found, return a leaf node with majority class
            majority_class = np.argmax(np.bincount(y))
            return {'value': majority_class}

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    def fit(self, X, y):
        # Train the decision tree
        self.features = set(range(X.shape[1]))
        self.tree = self._grow_tree(X, y, depth=0)

    def _predict_instance(self, x, node):
        # Recursive function to predict a single instance
        if 'value' in node:
            return node['value']

        if x[node['feature']] <= node['threshold']:
            return self._predict_instance(x, node['left'])
        else:
            return self._predict_instance(x, node['right'])

    def predict(self, X):
        # Predict for multiple instances
        if set(range(X.shape[1])) != self.features:
            raise ValueError("The number or order of features in the test set does not match the training set.")

        return [self._predict_instance(x, self.tree) for x in X]


class RandomForest:
    def __init__(self, num_trees=3, max_depth=None):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.num_trees):
            tree = DecisionTree(max_depth=self.max_depth, random_features=True)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        predictions = [tree.predict(X) for tree in self.trees]
        # Voting for classification (mode)
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)



In [40]:
train_df

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,label
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7489,0,82,9,59,56,34,41,0,10,30,3,67,42,96,100,100,5
7490,49,100,0,70,24,56,100,65,86,85,44,77,21,38,6,0,4
7491,100,98,60,100,24,87,3,58,35,51,58,26,36,0,0,5,5
7492,59,65,91,100,84,96,72,50,51,8,0,0,45,1,100,0,1


In [113]:
X = np.array(train_df.drop("label",axis=1))

Y = np.array(train_df['label'])
X_test=np.array(train_df.drop("label",axis=1))

In [97]:
import numpy as np




# class RandomForestnew:
#     def __init__(self, num_trees=3, max_depth=None):
#         self.num_trees = num_trees
#         self.max_depth = max_depth
#         self.trees = []

#     def fit(self, X, y):
#         for _ in range(self.num_trees):
#             tree = DecisionTree(max_depth=self.max_depth, random_features=True)
#             tree.fit(X, y)
#             self.trees.append(tree)

#     def _majority_vote(self, predictions):
#         return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

#     def predict(self, X):
#         predictions = np.array([tree.predict(X) for tree in self.trees])
#         return self._majority_vote(predictions)


In [98]:
# forest15 = RandomForestnew(num_trees=15, max_depth=None)
# forest15.fit(X, Y)

In [114]:

class DecisionTree:
    def __init__(self, max_depth=None, random_features=False):
        self.max_depth = max_depth
        self.tree = None
        self.random_features = random_features
        self.features = None  # Store the features used during training

    def _calculate_entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Add epsilon to avoid log(0)
        return entropy

    def _calculate_information_gain(self, X, y, feature, threshold):
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_entropy = self._calculate_entropy(y[left_mask])
        right_entropy = self._calculate_entropy(y[right_mask])

        total_entropy = self._calculate_entropy(y)
        information_gain = total_entropy - (np.sum(left_mask) / len(y) * left_entropy +
                                           np.sum(right_mask) / len(y) * right_entropy)

        return information_gain

    def _find_best_split(self, X, y):
        m, n = X.shape
        best_feature = None
        best_threshold = None
        max_information_gain = -np.inf

        if self.random_features:
            features_subset = np.random.choice(n, int(np.sqrt(n)), replace=False)
        else:
            features_subset = range(n)

        for feature in features_subset:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                information_gain = self._calculate_information_gain(X, y, feature, threshold)

                if information_gain > max_information_gain:
                    max_information_gain = information_gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _grow_tree(self, X, y, depth):
        unique_classes = np.unique(y)

        if len(unique_classes) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            return {'value': unique_classes[0]}

        best_feature, best_threshold = self._find_best_split(X, y)

        if best_feature is None:
            majority_class = np.argmax(np.bincount(y))
            return {'value': majority_class}

        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        left_subtree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        return {'feature': best_feature, 'threshold': best_threshold,
                'left': left_subtree, 'right': right_subtree}

    def fit(self, X, y):
        self.features = set(range(X.shape[1]))
        self.tree = self._grow_tree(X, y, depth=0)

    def _predict_instance(self, x, node):
        if 'value' in node:
            return node['value']

        if x[node['feature']] <= node['threshold']:
            return self._predict_instance(x, node['left'])
        else:
            return self._predict_instance(x, node['right'])

    def predict(self, X):
        if set(range(X.shape[1])) != self.features:
            raise ValueError("The number or order of features in the test set does not match the training set.")

        return [self._predict_instance(x, self.tree) for x in X]
class RandomForest:
    def __init__(self, num_trees=3, max_depth=None):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.num_trees):
            tree = DecisionTree(max_depth=self.max_depth)
            indices = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X[indices], y[indices])
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=0, arr=predictions)

In [117]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a dummy dataset
# X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



X = np.array(train_df.drop("label",axis=1))

Y = np.array(train_df['label'])
X_test=np.array(train_df.drop("label",axis=1))

# Example usage for forest3
# forest3 = RandomForest(num_trees=3, max_depth=None)
# forest3.fit(X_train, y_train)
# predictions_forest3 = forest3.predict(X_test)

# # Example usage for forest15
# forest3 = RandomForest(num_trees=3, max_depth=None)
# forest3.fit(X, Y)
# predictions_forest15 = forest15.predict(X_test)

random = DecisionTree(max_depth=None, random_features=True)
random.fit(X,Y)



# Evaluate the performance
# accuracy_forest3 = accuracy_score(y_test, predictions_forest3)
# accuracy_forest15 = accuracy_score(y_test, predictions_forest15)

# print(f'Accuracy for forest3: {accuracy_forest3:.4f}')
# print(f'Accuracy for forest15: {accuracy_forest15:.4f}')

TypeError: Cannot cast array data from dtype('O') to dtype('int64') according to the rule 'safe'

In [118]:
import numpy as np

class DecisionNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # Only for leaf nodes


class RandomizedDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1 or (self.max_depth is not None and depth == self.max_depth):
            return DecisionNode(value=self._most_common_class(y))

        # Random feature selection
        feature_indices = np.random.choice(num_features, int(np.sqrt(num_features)), replace=False)
        best_feature, best_threshold = self._best_split(X, y, feature_indices)

        if best_feature is None:
            return DecisionNode(value=self._most_common_class(y))

        # Split the dataset
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices

        # Recursive call to grow left and right subtrees
        left_tree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return DecisionNode(feature_index=best_feature, threshold=best_threshold,
                             left=left_tree, right=right_tree)

    def _best_split(self, X, y, feature_indices):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        for feature_index in feature_indices:
            thresholds, classes = self._find_splits(X[:, feature_index], y)
            for threshold, left_classes, right_classes in zip(thresholds, classes['left'], classes['right']):
                gini = self._gini_impurity(y, left_classes, right_classes)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def _find_splits(self, feature, y):
        sorted_indices = np.argsort(feature)
        feature_sorted = feature[sorted_indices]
        y_sorted = y[sorted_indices]

        boundaries = np.where(y_sorted[:-1] != y_sorted[1:])[0] + 0.5
        thresholds = feature_sorted[boundaries]
        classes = {'left': [], 'right': []}

        for threshold in thresholds:
            left_classes = y_sorted[feature_sorted <= threshold]
            right_classes = y_sorted[feature_sorted > threshold]
            classes['left'].append(left_classes)
            classes['right'].append(right_classes)

        return thresholds, classes

    def _gini_impurity(self, y, left_classes, right_classes):
        p_left = len(left_classes) / len(y)
        p_right = len(right_classes) / len(y)

        gini_left = 1 - np.sum([(np.sum(left_classes == c) / len(left_classes)) ** 2 for c in np.unique(left_classes)])
        gini_right = 1 - np.sum([(np.sum(right_classes == c) / len(right_classes)) ** 2 for c in np.unique(right_classes)])

        gini_impurity = p_left * gini_left + p_right * gini_right
        return gini_impurity

    def _most_common_class(self, y):
        unique_classes, counts = np.unique(y, return_counts=True)
        return unique_classes[np.argmax(counts)]

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._predict_tree(x, node.left)
        else:
            return self._predict_tree(x, node.right)


# Example usage
# Assuming X_train and y_train are your training data and labels
randomized_tree = RandomizedDecisionTree(max_depth=None)
randomized_tree.fit(X, Y)
predictions_randomized_tree = randomized_tree.predict(X_test)


IndexError: arrays used as indices must be of integer (or boolean) type

In [123]:
Y

array(['1', '1', '1', '1', '3', '1', '3', '2', '1', '1', '4', '7', '3',
       '4', '4', '7', '7', '7', '2', '4', '7', '7', '7', '2', '7', '7',
       '1', '7', '7', '7', '1', '3', '2', '1', '7', '1', '2', '2', '1',
       '7', '1', '2', '1', '2', '7', '2', '2', '1', '3', '1', '2', '3',
       '7', '2', '2', '1', '7', '3', '8', '2', '7', '5', '7', '2', '7',
       '2', '1', '2', '2', '7', '1', '2', '2', '7', '7', '3', '3', '7',
       '7', '3', '2', '1', '2', '2', '7', '6', '5', '1', '2', '2', '4',
       '2', '2', '5', '5', '2', '7', '2', '7', '5', '1', '7', '7', '7',
       '1', '7', '8', '1', '2', '7', '7', '2', '1', '7', '7', '7', '7',
       '5', '1', '7', '2', '3', '2', '1', '9', '3', '7', '2', '2', '2',
       '8', '4', '7', '4', '2', '2', '2', '2', '2', '5', '2', '1', '7',
       '2', '1', '8', '2', '2', '1', '7', '2', '2', '1', '7', '3', '2',
       '3', '1', '2', '2', '3', '9', '1', '2', '2', '1', '6', '2', '7',
       '7', '2', '5', '7', '5', '2', '7', '2', '1', '7', '2', '1

In [131]:
import numpy as np

# Generate dummy data
np.random.seed(42)
X_train = np.random.rand(100, 5)
y_train = np.random.choice([0, 1], size=100)

X_test = np.random.rand(20, 5)
y_test = np.random.choice([0, 1], size=20)

class DecisionNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class RandomizedDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        if len(unique_classes) == 1 or (self.max_depth is not None and depth == self.max_depth):
            return DecisionNode(value=self._most_common_class(y))

        feature_indices = np.random.choice(num_features, int(np.sqrt(num_features)), replace=False)
        best_feature, best_threshold = self._best_split(X, y, feature_indices)

        if best_feature is None:
            return DecisionNode(value=self._most_common_class(y))

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices

        left_tree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return DecisionNode(feature_index=best_feature, threshold=best_threshold,
                             left=left_tree, right=right_tree)

    def _best_split(self, X, y, feature_indices):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        for feature_index in feature_indices:
            thresholds, classes = self._find_splits(X[:, feature_index], y)
            for threshold, left_classes, right_classes in zip(thresholds, classes['left'], classes['right']):
                gini = self._gini_impurity(y, left_classes, right_classes)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def _find_splits(self, feature, y):
        sorted_indices = np.argsort(feature)
        feature_sorted = feature[sorted_indices]
        y_sorted = y[sorted_indices]

        boundaries = np.where(y_sorted[:-1] != y_sorted[1:])[0] + 0.5
        thresholds = feature_sorted[boundaries]
        classes = {'left': [], 'right': []}

        for threshold in thresholds:
            left_classes = y_sorted[feature_sorted <= threshold]
            right_classes = y_sorted[feature_sorted > threshold]
            classes['left'].append(left_classes)
            classes['right'].append(right_classes)

        return thresholds, classes

    def _gini_impurity(self, y, left_classes, right_classes):
        p_left = len(left_classes) / len(y)
        p_right = len(right_classes) / len(y)

        gini_left = 1 - np.sum([(np.sum(left_classes == c) / len(left_classes)) ** 2 for c in np.unique(left_classes)])
        gini_right = 1 - np.sum([(np.sum(right_classes == c) / len(right_classes)) ** 2 for c in np.unique(right_classes)])

        gini_impurity = p_left * gini_left + p_right * gini_right
        return gini_impurity

    def _most_common_class(self, y):
        unique_classes, counts = np.unique(y, return_counts=True)
        return unique_classes[np.argmax(counts)]

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._predict_tree(x, node.left)
        else:
            return self._predict_tree(x, node.right)

# Example usage
randomized_tree = RandomizedDecisionTree(max_depth=None)
randomized_tree.fit(X, Y)
predictions_randomized_tree = randomized_tree.predict(X_test)

print("Predictions:", predictions_randomized_tree)


IndexError: arrays used as indices must be of integer (or boolean) type

In [136]:
import numpy as np
class DecisionTree:
    def __init__(self, max_depth=None, randomized=False):
        self.max_depth = max_depth
        self.randomized = randomized
        self.tree = None

    def fit(self, X, y):
        self.tree = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        if self.max_depth is not None and depth == self.max_depth:
            return {'class': self._most_common_class(y)}

        # Random feature selection
        if self.randomized:
            feature_index = int(np.random.choice(num_features, size=1, replace=False)[0])
        else:
            feature_index = self._best_feature(X, y)

        thresholds, classes = self._find_splits(X, y, feature_index)

        if not thresholds:
            return {'class': self._most_common_class(y)}

        best_threshold = thresholds[np.argmax([self._information_gain(y, left, right) for left, right in classes])]

        # Split the dataset
        left_indices = X[:, feature_index] <= best_threshold
        right_indices = ~left_indices

        # Recursive call to grow left and right subtrees
        left_tree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return {'feature_index': feature_index, 'threshold': best_threshold,
                'left': left_tree, 'right': right_tree}


    def _best_feature(self, X, y):
        information_gains = [self._information_gain(y, X[:, i]) for i in range(X.shape[1])]
        return np.argmax(information_gains)

    def _find_splits(self, X, y, feature_index):
        sorted_indices = np.argsort(X[:, feature_index])
        feature_sorted = X[sorted_indices, feature_index]
        y_sorted = y[sorted_indices]

        boundaries = np.where(y_sorted[:-1] != y_sorted[1:])[0] + 0.5
        thresholds = feature_sorted[boundaries]
        classes = {'left': [], 'right': []}

        for threshold in thresholds:
            left_classes = y_sorted[feature_sorted <= threshold]
            right_classes = y_sorted[feature_sorted > threshold]
            classes['left'].append(left_classes)
            classes['right'].append(right_classes)

        return thresholds, classes

    def _information_gain(self, y, left_classes, right_classes):
        p_parent = len(y) / len(left_classes)
        p_left = len(left_classes) / len(y)
        p_right = len(right_classes) / len(y)

        entropy_parent = self._entropy(y)
        entropy_left = self._entropy(left_classes)
        entropy_right = self._entropy(right_classes)

        information_gain = entropy_parent - (p_left * entropy_left + p_right * entropy_right)
        return information_gain

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities))

    def _most_common_class(self, y):
        unique_classes, counts = np.unique(y, return_counts=True)
        return unique_classes[np.argmax(counts)]

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, tree):
        if 'class' in tree:
            return tree['class']
        if x[tree['feature_index']] <= tree['threshold']:
            return self._predict_tree(x, tree['left'])
        else:
            return self._predict_tree(x, tree['right'])

# Example usage
# Assuming X_train and y_train are your training data and labels


In [137]:
import numpy as np

# Generate dummy data
np.random.seed(42)
X_train = np.random.rand(100, 2)  # 100 samples, 2 features
y_train = np.random.randint(0, 2, size=100)  # Binary classification labels

X_test = np.random.rand(20, 2)  # 20 samples for testing
y_test = np.random.randint(0, 2, size=20)  # Corresponding labels

# Example usage
tree = DecisionTree(max_depth=None, randomized=True)
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

# Evaluate the performance
accuracy = np.mean(predictions == y_test)
print(f'Accuracy: {accuracy:.4f}')


IndexError: arrays used as indices must be of integer (or boolean) type

In [138]:
import numpy as np

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of the feature to split on
        self.threshold = threshold  # Threshold value for splitting
        self.left = left  # Left subtree
        self.right = right  # Right subtree
        self.value = value  # Class label for leaf nodes

def entropy(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return -np.sum(probabilities * np.log2(probabilities + 1e-10))

def information_gain(X, y, feature_index, threshold):
    # Calculate parent entropy
    parent_entropy = entropy(y)

    # Split the dataset
    left_mask = X[:, feature_index] <= threshold
    right_mask = ~left_mask

    # Calculate child entropies
    left_entropy = entropy(y[left_mask])
    right_entropy = entropy(y[right_mask])

    # Calculate information gain
    n_left = np.sum(left_mask)
    n_right = np.sum(right_mask)
    total_instances = len(y)
    information_gain_value = parent_entropy - (n_left / total_instances) * left_entropy - (n_right / total_instances) * right_entropy

    return information_gain_value

def random_split(X, y):
    num_features = X.shape[1]
    feature_index = np.random.choice(num_features)  # Randomly select a feature
    feature_values = X[:, feature_index]

    unique_values = np.unique(feature_values)
    if len(unique_values) == 1:
        # If all values are the same, cannot split further
        return None, None

    # Randomly select a threshold from unique feature values
    threshold = np.random.choice(unique_values)

    return feature_index, threshold

def build_tree(X, y, depth=0, max_depth=None):
    if depth == max_depth or len(np.unique(y)) == 1:
        # If max depth reached or pure node, create a leaf node
        return Node(value=np.argmax(np.bincount(y)))

    feature_index, threshold = random_split(X, y)

    if feature_index is not None:
        # Split the dataset based on the randomly selected attribute and threshold
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask

        left_subtree = build_tree(X[left_mask], y[left_mask], depth + 1, max_depth)
        right_subtree = build_tree(X[right_mask], y[right_mask], depth + 1, max_depth)

        return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

    else:
        # If unable to split further, create a leaf node
        return Node(value=np.argmax(np.bincount(y)))

# Example usage:
# Assume X and y are your feature matrix and target vector in numpy format
# X and y should be of the shape (number of samples, number of features) and (number of samples,) respectively.

# Example:
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
y = np.array([0, 0, 1, 1])

# Build the tree
tree = build_tree(X, y, max_depth=3)

# You can now use the 'tree' object for making predictions on new data.


ValueError: attempt to get argmax of an empty sequence

In [141]:
import numpy as np

class Node:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.prediction = self.get_prediction(labels)

    def get_prediction(self, labels):
        unique_labels, counts = np.unique(labels, return_counts=True)
        index = np.argmax(counts)
        return unique_labels[index]

def information_gain(left_labels, right_labels):
    total_count = len(left_labels) + len(right_labels)
    left_weight = len(left_labels) / total_count
    right_weight = len(right_labels) / total_count

    entropy_before = calculate_entropy(left_labels + right_labels)
    entropy_after = left_weight * calculate_entropy(left_labels) + right_weight * calculate_entropy(right_labels)

    return entropy_before - entropy_after

def calculate_entropy(labels):
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Adding small epsilon to avoid log(0)
    return entropy

def build_tree(data, labels, max_depth):
    if max_depth == 0 or len(set(labels)) == 1:
        return Node(data, labels)

    num_features = data.shape[1]
    feature_indices = np.random.choice(num_features, int(np.sqrt(num_features)), replace=False)

    best_feature_index = None
    best_threshold = None
    best_info_gain = -1

    for feature_index in feature_indices:
        thresholds = np.unique(data[:, feature_index])
        for threshold in thresholds:
            left_mask = data[:, feature_index] <= threshold
            right_mask = ~left_mask

            left_labels = labels[left_mask]
            right_labels = labels[right_mask]

            current_info_gain = information_gain(left_labels, right_labels)

            if current_info_gain > best_info_gain:
                best_info_gain = current_info_gain
                best_feature_index = feature_index
                best_threshold = threshold

    if best_info_gain == 0:
        return Node(data, labels)

    left_mask = data[:, best_feature_index] <= best_threshold
    right_mask = ~left_mask

    left_data = data[left_mask]
    left_labels = labels[left_mask]
    right_data = data[right_mask]
    right_labels = labels[right_mask]

    node = Node(data, labels)
    node.feature_index = best_feature_index
    node.threshold = best_threshold
    node.left = build_tree(left_data, left_labels, max_depth - 1)
    node.right = build_tree(right_data, right_labels, max_depth - 1)

    return node

def predict(tree, sample):
    if tree.left is None and tree.right is None:
        return tree.prediction

    if sample[tree.feature_index] <= tree.threshold:
        return predict(tree.left, sample)
    else:
        return predict(tree.right, sample)

# Example usage:
# Assuming you have a dataset `X` and corresponding labels `y`
# max_depth is the maximum depth of the tree
# tree = build_tree(X, y, max_depth)

# To make predictions:
# sample = np.array([feature1_value, feature2_value, ...])
# prediction = predict(tree, sample)


In [142]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Example usage of the random decision tree
max_depth = 3
tree = build_tree(X_train, y_train, max_depth)

# Make predictions on the test set
predictions = np.array([predict(tree, sample) for sample in X_test])

# Evaluate accuracy
accuracy = np.mean(predictions == y_test)
print(f"Accuracy on the test set: {accuracy:.2f}")


ValueError: operands could not be broadcast together with shapes (2,) (798,) 

In [145]:
import numpy as np

class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature        # Index of feature to split on
        self.threshold = threshold    # Threshold value for the feature
        self.left = left              # Left subtree
        self.right = right            # Right subtree
        self.value = value            # Class label for leaf nodes

class DecisionTree:
    def __init__(self, max_depth=None, randomized=False):
        self.max_depth = max_depth
        self.randomized = randomized
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _calculate_entropy(self, y):
        # Calculate entropy of a set of labels
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Add small epsilon to avoid log(0)
        return entropy

    def _calculate_information_gain(self, X, y, feature, threshold):
        # Calculate information gain for a given split
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask
        left_entropy = self._calculate_entropy(y[left_mask])
        right_entropy = self._calculate_entropy(y[right_mask])
        total_entropy = self._calculate_entropy(y)
        information_gain = total_entropy - (
            (len(y[left_mask]) / len(y)) * left_entropy +
            (len(y[right_mask]) / len(y)) * right_entropy
        )
        return information_gain

    def _find_best_split(self, X, y):
        # Find the best split (feature and threshold) for a node
        features = X.shape[1]
        best_feature, best_threshold, best_gain = None, None, -1

        if self.randomized:
            # Randomly select a feature
            random_feature = np.random.randint(features)
            thresholds = np.unique(X[:, random_feature])
        else:
            # Iterate through all features and thresholds
            for feature in range(features):
                thresholds = np.unique(X[:, feature])
                for threshold in thresholds:
                    gain = self._calculate_information_gain(X, y, feature, threshold)
                    if gain > best_gain:
                        best_feature, best_threshold, best_gain = feature, threshold, gain

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            # Create a leaf node
            return TreeNode(value=np.argmax(np.bincount(y)))

        feature, threshold = self._find_best_split(X, y)

        if feature is not None:
            # Split the dataset
            left_mask = X[:, feature] <= threshold
            right_mask = ~left_mask
            left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
            right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

            return TreeNode(feature=feature, threshold=threshold, left=left_subtree, right=right_subtree)

    def predict(self, X):
        # Make predictions for a set of instances
        predictions = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            node = self.tree
            while node.left and node.right:
                if X[i, node.feature] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            predictions[i] = node.value
        return predictions.astype(int)

class RandomForest:
    def __init__(self, n_trees=3, **tree_params):
        self.n_trees = n_trees
        self.tree_params = tree_params
        self.forest = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            tree = DecisionTree(**self.tree_params)
            indices = np.random.choice(len(X), len(X), replace=True)
            tree.fit(X[indices], y[indices])
            self.forest.append(tree)

    def predict(self, X):
        # Make predictions using each tree and combine the results
        predictions = np.array([tree.predict(X) for tree in self.forest])
        return np.argmax(np.bincount(predictions, axis=0), axis=0)

# Example usage:
# Assuming X_train, y_train, X_test are your training features, training labels, and test features
# dt = DecisionTree(max_depth=5, randomized=False)
# dt.fit(X_train, y_train)
# predictions = dt.predict(X_test)

# rf = RandomForest(n_trees=15, max_depth=5, randomized=True)
# rf.fit(X_train, y_train)
# predictions = rf.predict(X_test)


In [146]:
# Dummy data generation
np.random.seed(42)
X_train = np.random.rand(100, 2)  # 100 samples, 2 features
y_train = np.random.randint(0, 2, 100)  # Binary labels

X_test = np.random.rand(20, 2)  # 20 samples for testing

# Using the DecisionTree
dt = DecisionTree(max_depth=3, randomized=False)
dt.fit(X_train, y_train)
dt_predictions = dt.predict(X_test)

print("Decision Tree Predictions:")
print(dt_predictions)

# Using the RandomForest
rf = RandomForest(n_trees=3, max_depth=3, randomized=True)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)

print("\nRandom Forest Predictions:")
print(rf_predictions)


Decision Tree Predictions:
[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1]


AttributeError: 'NoneType' object has no attribute 'left'

In [155]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class DecisionNode:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

class DecisionTree:
    def __init__(self, max_depth=None, randomized=False):
        self.max_depth = max_depth
        self.randomized = randomized

    def fit(self, X, y):
        self.num_classes = len(np.unique(y))
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        unique_classes, counts = np.unique(y, return_counts=True)

        # Create a leaf node if only one class or maximum depth reached
        if len(unique_classes) == 1 or (self.max_depth is not None and depth == self.max_depth):
            return DecisionNode(value=unique_classes[np.argmax(counts)])

        # Randomized feature selection
        if self.randomized:
            feature_indices = np.random.choice(num_features, size=num_features, replace=False)
        else:
            feature_indices = range(num_features)

        # Find the best split
        best_feature, best_threshold = self._find_best_split(X, y, feature_indices)

        # Create sub-datasets for the left and right subtrees
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices

        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return DecisionNode(feature_index=best_feature, threshold=best_threshold,
                            left=left_subtree, right=right_subtree)

    def _find_best_split(self, X, y, feature_indices):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        for feature in feature_indices:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = ~left_indices

                gini = self._calculate_gini(y[left_indices]) + self._calculate_gini(y[right_indices])
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _calculate_gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._predict_tree(x, node.left)
        else:
            return self._predict_tree(x, node.right)

class RandomForest:
    def __init__(self, n_trees=15, max_depth=None, randomized=True):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.randomized = randomized
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, randomized=self.randomized)
            indices = np.random.choice(len(X), size=len(X), replace=True)
            tree.fit(X[indices], y[indices])
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

# Example usage with the Iris dataset
train_df=train_df.astype(float)
X=np.array(train_df.drop("label",axis=1))
Y=np.array(train_df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Decision Tree
# dt = DecisionTree()
# dt.fit(X_train, y_train)
# y_pred_dt = dt.predict(X_test)
# accuracy_dt = accuracy_score(y_test, y_pred_dt)
# print(f"Decision Tree Accuracy: {accuracy_dt:.2f}")

# Random Forest
rf = RandomForest(n_trees=15)
rf.fit(X_train, y_train)
X_test=X_test.astype(int)
y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.2f}")


TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'

In [148]:
X

array([[ 1.12510039,  1.17812384,  0.49351604, ...,  1.35732466,
         0.9660408 , -1.98113862],
       [-0.56464086,  3.6386291 , -1.52241469, ..., -0.89025442,
         1.43882638, -3.82874758],
       [ 0.51631285,  2.16542633, -0.62848571, ..., -1.95817543,
        -0.34880315, -1.8041241 ],
       ...,
       [ 1.65015307, -0.69216458, -2.04920577, ..., -1.30257748,
        -1.28550452,  3.32856934],
       [-1.18660302, -1.41459786, -0.12151968, ..., -1.42146469,
        -0.02833985,  3.41393228],
       [ 0.78867591, -0.22254747,  0.32856985, ..., -1.29103957,
        -2.33817245,  2.03602059]])

In [ ]:
import numpy as np

class RandomForest:
    def __init__(self, n_trees, max_depth=None, randomized=False):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.randomized = randomized
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, randomized=self.randomized)
            indices = np.random.choice(len(X), size=len(X), replace=True)
            tree.fit(X[indices], y[indices])
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

# Example usage with 3 trees
np.random.seed(42)  # For reproducibility
rf_3_trees = RandomForest(n_trees=3, max_depth=None, randomized=True)
rf_3_trees.fit(X_train, y_train)
predictions_3_trees = rf_3_trees.predict(X_test)

# Example usage with 15 trees
np.random.seed(42)  # For reproducibility
rf_15_trees = RandomForest(n_trees=15, max_depth=None, randomized=True)
rf_15_trees.fit(X_train, y_train)
predictions_15_trees = rf_15_trees.predict(X_test)
